### Import Libraries

In [1]:
import requests
import tabula
import pandas as pd

### Fetch PDF links

In [2]:
# 23s running time
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Referer': 'https://pdaandacollo.teck.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
}

response = requests.get('https://pdaandacollo.teck.com/services/api/DirectoryBrowser',headers=headers)

In [3]:
root = response.json()

branch_acierto = root[14] # Registro Acierto de Tronadura Teck CDA

base_url = 'https://pdaandacollo.teck.com/services/api/FileDownload/?filePath=\\'
years = ['2023','2024']
months_2023 = ['08. August','09. Septiembre','10. Octubre','11. Noviembre','12. Diciembre']

url_pdfs = [
    base_url + tronadura['Path'].replace(' ', '%20')
    for year in branch_acierto['Nodes'] if year['Name'] in years
    for month in year['Nodes']
    if (year['Name'] == '2024' or (year['Name'] == '2023' and month['Name'] in months_2023))
    for tronadura in month['Nodes']
]

### Convert PDF to DataFrame

In [4]:
# 5m 10s running time
tables = []

for url_pdf in url_pdfs:

    # print(url_pdf)
    
    table = tabula.read_pdf(url_pdf, pages=1, multiple_tables=True)[-1]

    table.insert(0, 'Date', url_pdf.replace('%20','')[-14:-4])
    table.insert(0, 'URL', url_pdf) 

    tables.append(table)


### Concatenate and Clean Tables

In [5]:
tables_clean = []

for table in tables:
    table.ffill(inplace=True)
    # table[table['Unnamed: 2'].str.contains(r'\d', na=False)]
    table = table.iloc[5:,:7].dropna()
    tables_clean.append(table)

In [33]:
# raw tables concatenated
tables_concatenated = pd.concat(tables_clean)

# reset index
tables_concatenated.reset_index(drop=True, inplace=True)

# filter irrelevant rows
tables_concatenated = tables_concatenated[~tables_concatenated['Unnamed: 0'].str.contains('N°')]

# delete duplicates
tables_concatenated.drop_duplicates(subset=['Date','Unnamed: 1','Unnamed: 2','Unnamed: 3'], inplace=True)

# filter out rows that do not have ':' twice in row
mask = tables_concatenated.map(lambda x: str(x).count(':'))
row_sums = mask.sum(axis=1)
filtered_df = tables_concatenated[row_sums >= 2]

# New Column: Hora de Ejecucion
filtered_df['all_columns_joined'] = filtered_df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
pattern = r'\b(\d{1,2}:\d{1,2})\b'
filtered_df['Hora de ejecución'] = filtered_df['all_columns_joined'].str.extractall(pattern).groupby(level=0).last()

# rename column name
filtered_df.rename({
    'Unnamed: 1':'Fase, banco, y malla programada',
    'Unnamed: 2':'Estado / Motivo de suspensión'
    }, axis=1, inplace=True)

# replace dots with a dash for Date column
filtered_df['Date'] = filtered_df['Date'].str.replace('.', '-').copy()

# select columns
filtered_df = filtered_df[['Date','Fase, banco, y malla programada','Estado / Motivo de suspensión','Hora de ejecución','URL']]


### Save table as CSV

In [30]:
filtered_df.to_csv('pdaandacollo-teck.csv', encoding='iso-8859-1', index=False)